In [ ]:
import identification as ob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import control as con
import warnings
import time
start_time = list(time.localtime())[5] #returns the local time at the start of execution. Can be used to calculate the execution time manually
%pylab inline
warnings.filterwarnings('ignore') #ignores all the warnings that may arise during runtime

###### Reading all the data

In [ ]:
control_output_1 = 'data/2017-11-03_14-25-18_control_output.log'
task_velocity_1  = 'data/2017-11-03_14-25-19_task_vel.log'

control_output_2 = 'data/2017-11-03_14-26-43_control_output.log'
task_velocity_2  = 'data/2017-11-03_14-26-43_task_vel.log'

control_output_3 = 'data/2017-11-03_14-27-47_control_output.log'
task_velocity_3  = 'data/2017-11-03_14-27-47_task_vel.log'

control_output_4 = 'data/2017-11-03_14-30-43_control_output.log'
task_velocity_4  = 'data/2017-11-03_14-30-43_task_vel.log'

df_ist_soll_1 = ob.read_data(control_output_1, task_velocity_1)
df_ist_soll_2 = ob.read_data(control_output_2, task_velocity_2)
df_ist_soll_3 = ob.read_data(control_output_3, task_velocity_3)
df_ist_soll_4 = ob.read_data(control_output_4, task_velocity_4)
#df_ist_soll_1.plot(style = '-', drawstyle = "steps")

###### Removing all zeros & the negative trend and reformatting the data in accordance with a unit step response

In [ ]:
#data1
xin_1, yout_1, t_1 = ob.strip_multiply(df_ist_soll_1)
#plt.plot(t_1,yout_1)
#plt.plot(t_1,xin_1)


#data2
xin_2, yout_2, t_2 = ob.strip_multiply(df_ist_soll_2)  
#plt.plot(t_2,yout_2)
#plt.plot(t_2,xin_2)


#data3-data with a special case trend
df_ist_soll_3          = df_ist_soll_3[(df_ist_soll_3.x_ist > 0)]
#the following code identifies the decreasing trend in x_ist values and removes those values from the dataframe
df_ist_soll_3['trend'] = np.sign(df_ist_soll_3['x_ist'].rolling(window = 5).mean().diff().fillna(0)).map({0:'FLAT', 1:'UP', -1:'DOWN'})
reversed_array = list(df_ist_soll_3.trend.values)[::-1]
counter = 0
for i in range(0, len(reversed_array)):
    if reversed_array[i] == 'DOWN':
        counter = counter + 1
    else:
        break
length             = len(df_ist_soll_3)
df_ist_soll_3      = df_ist_soll_3.head(length - counter)
xin_3, yout_3, t_3 = ob.strip_multiply(df_ist_soll_3)  
#plt.plot(t_3,yout_3)
#plt.plot(t_3,xin_3)


#data4
xin_4, yout_4, t_4 = ob.strip_multiply(df_ist_soll_4)  
#plt.plot(t_4,yout_4)
#plt.plot(t_4,xin_4)


yout = [yout_1, yout_2, yout_3, yout_4]
t = [t_1, t_2, t_3, t_4]

###### Finding the fitted model and aic values using R 

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R 
library(stats) #need this for using the ar() function
library(forecast) #need this for using the fitted.values() function

In [ ]:
#The function returns the aic,mse and fitted values for a given output data and a given order 
def order_ar_R(ar_order, output):
    %R -i ar_order,output #data is inputed as a list vector. In a list vector, each element in the list is an array. No space can be provided after comma. If provided, it results in 'NameError: name '' is not defined' error.
    %R output                 = unlist(output)  #converts the list vector back into a single array. Need this step in latest R version 3.4.2.
    %R ar_system              = ar(output, method = "ols", order.max = ar_order)
    %R fitted_ar_with_missing = fitted.values(ar_system) #the lower values in the output data contributes to NA/missing values in the fitted values. This corresponds to the delay in the output data.
    %R -o fitted_ar_with_missing
    
    fitted_ar_without_missing = np.nan_to_num(fitted_ar_with_missing) #the missing values becomes nan values in Python and they are converted to 0. It becomes easier in finding the delay for our model.
    mse_ar = mean_squared_error(output,fitted_ar_without_missing)
        
    %R -i mse_ar
    %R output_length = length(output) 
    %R aic_ar        = (output_length * log(mse_ar)) + (2 * ar_order) + (output_length * dim(matrix(output))[2] * (log(2 * pi) + 1)) #result obtained from https://rdrr.io/cran/sysid/src/R/estpoly.R
    %R -o aic_ar,mse_ar
    
    return list(aic_ar), mse_ar, fitted_ar_without_missing

###### Finding the fitted model and aic values using Python 

In [ ]:
order_ar_P = ob.order_ar_P

###### MSE and AIC Dataframe for orders 1 to 10 along with fitted values

In [ ]:
#The function returns a dataframe that contains the aic,mse and order values from 1 to 10 along with the fitted values for each order 

def order_aic_mse_fit(yout):
    aic                  = []
    order                = []
    mse                  = []
    fit_array            = []
    aic_mse_fit_df       = [] 
    aic_mse_fit_df.append([])#2D array in which each element stores two floats(aic and mse) and an array(fit_array)
   
    for i in range(1,11):
        order.append(i)
        aic_mse_fit_df.append(order_ar_P(i,yout))
        aic.append(aic_mse_fit_df[i][0])
        mse.append(aic_mse_fit_df[i][1])
        fit_array.append(aic_mse_fit_df[i][2])
    
    df = pd.DataFrame(np.column_stack([order, aic, mse]),columns=['order', 'aic', 'mse']) #all variables are passed into the dataframe as type float by default  
    return df, fit_array
    

df_1, fit_array_1 = order_aic_mse_fit(yout_1)
df_2, fit_array_2 = order_aic_mse_fit(yout_2)
df_3, fit_array_3 = order_aic_mse_fit(yout_3)
df_4, fit_array_4 = order_aic_mse_fit(yout_4)

###### Selecting the best order and the corresponding aic, mse and fitted values for each data based on low mse value

In [ ]:
order_1, mse_ar_1, aic_ar_1 = list(df_1[df_1.mse == df_1.mse.min()].values[0]) #list function lists the df row having the least mse into a 1D array 
fitted_1                    = fit_array_1[int(order_1) - 1]#int function converts the order back to the integer value. Order-1 gives the corresponding index value 

order_2, mse_ar_2, aic_ar_2 = list(df_2[df_2.mse == df_2.mse.min()].values[0])
fitted_2                    = fit_array_2[int(order_2) - 1]

order_3, mse_ar_3, aic_ar_3 = list(df_3[df_3.mse == df_3.mse.min()].values[0])
fitted_3                    = fit_array_3[int(order_3) - 1]

order_4, mse_ar_4, aic_ar_4 = list(df_4[df_4.mse == df_4.mse.min()].values[0])
fitted_4                    = fit_array_4[int(order_4) - 1]

#plt.plot(t_4,yout_4,label='original')
#plt.plot(t_4,fitted_4,label='model')
#plt.legend()

###### Smoothing all the model outputs

In [ ]:
smooth_11 = ob.smooth(fitted_1, 1)
smooth_21 = ob.smooth(fitted_2, 1)
smooth_31 = ob.smooth(fitted_3, 1)
smooth_41 = ob.smooth(fitted_4, 1)

smooth_12 = ob.smooth(fitted_1, 2)
smooth_22 = ob.smooth(fitted_2, 2)
smooth_32 = ob.smooth(fitted_3, 2)
smooth_42 = ob.smooth(fitted_4, 2)

#smooth_11 = ob.smooth(yout_1, 1)
#smooth_21 = ob.smooth(yout_2, 1)
#smooth_31 = ob.smooth(yout_3, 1)
#smooth_41 = ob.smooth(yout_4, 1)

#smooth_12 = ob.smooth(yout_1, 2)
#smooth_22 = ob.smooth(yout_2, 2)
#smooth_32 = ob.smooth(yout_3, 2)
#smooth_42 = ob.smooth(yout_4, 2)

#plt.plot(t_1, smooth_11, label='smoothed')
#plt.plot(t_1, fitted_1, label='fitted_values')
#plt.plot(t_1, yout_1, label='data')
#plt.legend()

###### PT1 and PT2 Modeling on all data 

In [ ]:
#youto,to are the yout and t outputs from the pt1 and pt2 system
tf_11, delay_11, youto_11, to_11 = ob.pt1(smooth_11, t_1)
tf_21, delay_21, youto_21, to_21 = ob.pt1(smooth_21, t_2)
tf_31, delay_31, youto_31, to_31 = ob.pt1(smooth_31, t_3)
tf_41, delay_41, youto_41, to_41 = ob.pt1(smooth_41, t_4)
tf_12, delay_12, youto_12, to_12 = ob.pt2(smooth_12, t_1)
tf_22, delay_22, youto_22, to_22 = ob.pt2(smooth_22, t_2)
tf_32, delay_32, youto_32, to_32 = ob.pt2(smooth_32, t_3)
tf_42, delay_42, youto_42, to_42 = ob.pt2(smooth_42, t_4)

youto1 = [youto_11, youto_21, youto_31, youto_41]
to1    = [to_11, to_21, to_31, to_41]
youto2 = [youto_12, youto_22, youto_32, youto_42]
to2    = [to_12, to_22, to_32, to_42]

#plt.plot(to_2,youto_2)
#plt.plot(T_2,yout_2)

###### State space representation 

In [ ]:
#the function returns the steady state(ss) parameters for a given transfer function
def ss(TF):
    ss_parameters = con.matlab.tf2ss(TF)
    return ss_parameters

ss_11 = ss(tf_11 * delay_11)
ss_21 = ss(tf_21 * delay_21)
ss_31 = ss(tf_31 * delay_31)
ss_41 = ss(tf_41 * delay_41)
ss_12 = ss(tf_12 * delay_12)
ss_22 = ss(tf_22 * delay_22)
ss_32 = ss(tf_32 * delay_32)
ss_42 = ss(tf_42 * delay_42)
ss1 = [ss_11, ss_21, ss_31, ss_41]
ss2 = [ss_12, ss_22, ss_32, ss_42]

###### Mean square comparison of each data and its model

In [ ]:
mse_11 = ob.mse(yout_1, t_1, youto_11, to_11)
mse_21 = ob.mse(yout_2, t_2, youto_21, to_21)
mse_31 = ob.mse(yout_3, t_3, youto_31, to_31)
mse_41 = ob.mse(yout_4, t_4, youto_41, to_41)
mse_12 = ob.mse(yout_1, t_1, youto_12, to_12)
mse_22 = ob.mse(yout_2, t_2, youto_22, to_22)
mse_32 = ob.mse(yout_3, t_3, youto_32, to_32)
mse_42 = ob.mse(yout_4, t_4, youto_42, to_42)
mse1 = [mse_11, mse_21, mse_31, mse_41]
mse2 = [mse_12, mse_22, mse_32, mse_42]

###### PT1 and PT2 Model response of all the dataset

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def display_pt1(x):
    plt.plot(to1[x-1], youto1[x-1], label = 'pt1 model')
    plt.plot(t[x-1], yout[x-1], label = 'data')
    plt.legend()
    print('Mean square error is:')
    print(mse1[x-1])
    return ss1[x-1]
def display_pt2(x):
    plt.plot(to2[x-1], youto2[x-1], label = 'pt2 model')
    plt.plot(t[x-1], yout[x-1], label = 'data')
    plt.legend()
    print('Mean square error is:')
    print(mse2[x-1])
    return ss2[x-1]
def selection(s):
    if s == 'pt1':
        #interact(display_pt1,x = widgets.IntSlider(min=1,max=4,step=1));
        interact(display_pt1, x = widgets.RadioButtons(options = [1, 2, 3, 4], value = 1, description = 'Data:', disabled = False));
    else:
        #interact(display_pt2,x = widgets.IntSlider(min=1,max=4,step=1));
        interact(display_pt2, x = widgets.RadioButtons(options = [1, 2, 3, 4], value = 1, description = 'Data:', disabled = False));

interact(selection, s = widgets.RadioButtons(options = ['pt1', 'pt2'], value = 'pt1', description = 'System:', disabled = False));

###### Selecting the best model parameters among the dataset 

In [ ]:
#the below loop calculates the model with the lowest mse among both pt1 and pt2 models
for i in range(0, len(mse1)):
    for j in range(0, len(mse2)):
        if(mse1[i] == min(mse1)):
            i_low_mse = i
        if(mse2[j] == min(mse2)):
            j_low_mse = j
            
if mse2[j_low_mse] < mse1[i_low_mse]:
    plt.plot(to2[j_low_mse], youto2[j_low_mse], label = 'pt2 model')
    plt.plot(t[j_low_mse], yout[j_low_mse], label = 'data')
    plt.legend()
    print('The best model is a pt2 system and is obtained from dataset:', j_low_mse + 1) #index value + 1 gives the best dataset 
    print('Mean square error is:', mse2[j_low_mse])
    print('The state space parameters are:')
    print(ss2[j_low_mse])
else:
    plt.plot(to1[i_low_mse], youto1[i_low_mse], label = 'pt2 model')
    plt.plot(t[i_low_mse], yout[i_low_mse], label = 'data')
    plt.legend()
    print('The best model is a pt1 system and is obtained from dataset:', i_low_mse + 1)
    print('Mean square error is:', mse1[i_low_mse])
    print('The state space parameters are:')
    print(ss1[i_low_mse])

In [ ]:
end_time = list(time.localtime())[5]
s=np.subtract(end_time, start_time)
print('Total execution time is ',abs(s),' seconds')